In [ ]:
import pandas as pd
import altair as alt

In [ ]:
mapping = pd.read_csv('../../02_exomiser_manuscript//GS_ID_mapping.csv')


In [ ]:
def filt_vs_unfilt_df(unfiltered_run_type, filtered_run_type, success_table, limit,domain):
    data = []
    to_plot = []
    big_change=[]
    plot3=[]

    successes = success_table[success_table['Variant_Level_noMOI_'+str(filtered_run_type)] == 'Variant_Present_noMOI']
    successes = success_table[success_table['Variant_Level_noMOI_'+str(unfiltered_run_type)] == 'Variant_Present_noMOI']

    for i, patient_row in success_table.iterrows():
        ID = patient_row['Dumb_ID']
        unfiltered_rank = patient_row['Variant_Level_noMOI_rank_' + unfiltered_run_type]
        filtered_rank = patient_row['Variant_Level_noMOI_rank_' + filtered_run_type]
        if filtered_rank == 'N/A':
            filtered_rank = 0
        difference = abs(unfiltered_rank - filtered_rank)
       
        if unfiltered_rank > limit and filtered_rank <= limit and filtered_rank >0:
            if unfiltered_rank >= 69:
                big_change.append(ID)
            else:
                to_plot.append(ID)
                # if difference >30:
                #     to_plot.append(ID)
        if unfiltered_rank <0:
            if unfiltered_rank != filtered_rank:
                if filtered_rank <= limit and filtered_rank > 0:
                    to_plot.append(ID)

        # if unfiltered_rank < filtered_rank:
        #     to_plot.append(ID)

        data.append([ID, unfiltered_rank, difference, str(unfiltered_run_type)])
        data.append([ID, filtered_rank, difference, str(filtered_run_type)])

    df = pd.DataFrame(data, columns=['ID', 'Rank', 'Diff', 'RunType'])
    print(len(to_plot) + len(big_change))

    to_plot = list(set(to_plot) - set(big_change))

    source = df[df['ID'].isin(big_change)].sort_values(by=['Diff', 'Rank'], ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    rank_changes = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',strokeWidth=1,opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='set1', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )

    source = df[df['ID'].isin(to_plot)].sort_values(by=['Rank'], ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    rank_changes_big = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',strokeWidth=1,opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='set1', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=250,
        width=150
    )

    source = df[df['ID'].isin(plot3)].sort_values(by='Diff', ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    

    return df, to_plot, big_change,rank_changes_big, rank_changes



## A: WES Exomiser

In [ ]:
success_table = pd.read_csv('../../02_exomiser_manuscript/fig_03/ES_genomiser_fig3_input_1.13.25.tsv', sep='\t')
RUN_TYPES = ['filtered_exomiser_default', 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL']
domain = ['','filtered_exomiser_default', 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL']

mapped= []
for i, row in success_table.iterrows():
    ID = row['ID']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
success_table['Dumb_ID'] = mapped

In [ ]:
data = []
to_plot = []

unfiltered_run_type = 'filtered_exomiser_default'
filtered_run_type = 'noN_filtered_15_85_human_revel_mvp_alphaM_spliceAI_noWL'
df,big_change, to_plot,rank_changes_es, rank_changes_big_es= filt_vs_unfilt_df(unfiltered_run_type,filtered_run_type,success_table, 10, domain)

es_exomiser_improvement = alt.hconcat(
                                      rank_changes_es.properties(height=300,width=200))
es_exomiser_improvement

## B: WGS Genomiser

In [ ]:
success_table=pd.read_csv("../../02_exomiser_manuscript/fig_06_supp_fig_16/GS_genomiser_fig4_input_12.16.24.tsv", sep='\t')
RUN_TYPES = ['default_genomiser_genomiser','noN_filtered_15_18_human_REMM_revel_mvp_alphaM_spliceAI_PSF_501_reg_filter_noWL_genomiser']
domain =['default_genomiser_genomiser','','noN_filtered_15_18_human_REMM_revel_mvp_alphaM_spliceAI_PSF_501_reg_filter_noWL_genomiser']
success_table=success_table.fillna(-10)

mapped= []
for i, row in success_table.iterrows():
    ID = row['ID']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
success_table['Dumb_ID'] = mapped

In [ ]:
def filt_vs_unfilt_df(unfiltered_run_type, filtered_run_type, success_table, limit,domain,order):
    data = []
    to_plot = []
    big_change=[]
    plot3=[]

    successes = success_table[success_table['Variant_Level_noMOI_'+str(filtered_run_type)] == 'Variant_Present_noMOI']
    successes = success_table[success_table['Variant_Level_noMOI_'+str(unfiltered_run_type)] == 'Variant_Present_noMOI']

    for i, patient_row in success_table.iterrows():
        ID = patient_row['Dumb_ID']
        unfiltered_rank = patient_row['Variant_Level_noMOI_rank_' + unfiltered_run_type]
        filtered_rank = patient_row['Variant_Level_noMOI_rank_' + filtered_run_type]
        if filtered_rank == 'N/A':
            filtered_rank = 0
        difference = abs(unfiltered_rank - filtered_rank)
       
        if unfiltered_rank > limit and filtered_rank <= limit and filtered_rank >0:
            if unfiltered_rank >= 69:
                big_change.append(ID)
            else:
                to_plot.append(ID)
                # if difference >30:
                #     to_plot.append(ID)
        if unfiltered_rank <0:
            if unfiltered_rank != filtered_rank:
                if filtered_rank <= limit and filtered_rank > 0:
                    to_plot.append(ID)

        # if unfiltered_rank < filtered_rank:
        #     to_plot.append(ID)

        data.append([ID, unfiltered_rank, difference, str(unfiltered_run_type)])
        data.append([ID, filtered_rank, difference, str(filtered_run_type)])

    df = pd.DataFrame(data, columns=['ID', 'Rank', 'Diff', 'RunType'])
    print(len(to_plot) + len(big_change))

    to_plot = list(set(to_plot) - set(big_change))

    source = df[(df['ID'].isin(big_change) & (df['Rank']>20))].sort_values(by=['Diff', 'Rank'], ascending=False)
    #order = list(source['ID'])
    print(len(source)/2)
    rank_changes = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',strokeWidth=1,opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)).scale(domain=order),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant',scale=alt.Scale(domain=[100,500])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='set1', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=100,
        width=300
    )
    source = df[(df['ID'].isin(to_plot)) | (((df['Rank']<=20) & (df['ID'].isin(big_change))) )].sort_values(by=['Rank'], ascending=False)
    #order = list(source['ID'])
    print(len(source)/2)
    rank_changes_big = alt.Chart(source).mark_circle(size=100, filled=True, stroke='black',strokeWidth=1,opacity=1).encode(
        x=alt.X('ID', sort=order,axis=alt.Axis(labelAngle=-60)).scale(domain=order),
        y=alt.Y('Rank', title='Exomiser Rank of Causative Variant'),#, scale=alt.Scale(domain=[0,29.1])),
        #size = 'Term_Diff',
        color=alt.Color('RunType',  scale=alt.Scale(scheme='set1', domain=domain), legend=alt.Legend(
        orient='top',
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ['RunType', 'ID', 'Rank', 'Diff']
    ).properties(
        height=200,
        width=300
    )

    source = df[df['ID'].isin(plot3)].sort_values(by='Diff', ascending=False)
    order = list(source['ID'])
    print(len(source)/2)
    

    return df, to_plot, big_change,rank_changes_big, rank_changes



In [ ]:
order =['UDN119_HADHA', 'UDN130_TH', 'UDN64_PLA2G6','UDN305_COLQ',
 'UDN305_COLQ',
 'UDN284_HADHB',
 'UDN284_HADHB',
 'UDN284_HADHB',
 'UDN284_HADHB',
 'UDN177_HGSNAT',
 'UDN177_HGSNAT',
 'UDN177_HGSNAT',
 'UDN177_HGSNAT',
 'UDN109_NDUFAF8',
 'UDN109_NDUFAF8',
 'UDN109_NDUFAF8',
 'UDN109_NDUFAF8',
 'UDN88_COL6A1',
 'UDN308_ITPR1',
 'UDN88_COL6A1',
 'UDN308_ITPR1',
 'UDN305_COLQ',
 'UDN305_COLQ']

In [ ]:
success_table=pd.read_csv("../../02_exomiser_manuscript/fig_06_supp_fig_16/GS_genomiser_fig4_input_12.16.24.tsv", sep='\t')
RUN_TYPES = ['default_genomiser_genomiser','noN_filtered_15_18_human_REMM_revel_mvp_alphaM_spliceAI_PSF_501_reg_filter_noWL_genomiser']
domain =['','default_genomiser_genomiser','noN_filtered_15_18_human_REMM_revel_mvp_alphaM_spliceAI_PSF_501_reg_filter_noWL_genomiser']
success_table=success_table.fillna(-5)

mapped= []
for i, row in success_table.iterrows():
    ID = row['ID']
    mapped_id = mapping[mapping['ID']==ID]['Dumb_ID'].item()
    mapped.append(mapped_id)
success_table['Dumb_ID'] = mapped

In [ ]:
data = []
to_plot = []

unfiltered_run_type = 'default_genomiser_genomiser'
filtered_run_type = 'noN_filtered_15_18_human_REMM_revel_mvp_alphaM_spliceAI_PSF_501_reg_filter_noWL_genomiser'
df,big_change, to_plot,rank_changes_gs_genomiser, rank_changes_big_gs_genomiser = filt_vs_unfilt_df(unfiltered_run_type,filtered_run_type,success_table, 10, domain,order)
# gs_genomiser_improvement = alt.hconcat(rank_changes_big_gs_genomiser,
#                                       rank_changes_gs_genomiser)
gs_genomiser_improvement = alt.vconcat(rank_changes_big_gs_genomiser, rank_changes_gs_genomiser, spacing=5).resolve_scale(x="shared")
gs_genomiser_improvement

In [ ]:
alt.hconcat(es_exomiser_improvement, gs_genomiser_improvement).resolve_scale(color='independent').resolve_scale(color='independent').configure_legend(labelLimit=0, labelFontSize=15, labelFont='arial').configure_axis(
    labelFontSize=13,
    titleFontSize=15,
    labelFont='arial').configure_scale(
    bandPaddingInner=0.9

)